In [1]:
%matplotlib inline

In [11]:

import cv2 as cv

#use thinned graph of corner
#iterate over each pixel
    #check neighboring pixels and count how many are "marked"
        #if num neighbors > 2 or num neighbors < 2
            #pixel = node
        #else
            #pixel = edge
#read 3D rbg image as 2D greyscale nparray
blobs = cv.imread(cv.samples.findFile("../Lake_Murray_Map_Skeletons/SE_corner_thinned.png"),0) 
# last parameter 0 = greyscale 1 = color -1 = unchanged 

#convert data from int to np bool 
blobs = blobs > 127 
#blobs is now a (540,960) 2 D array

print(blobs.shape)

ydim = blobs.shape[0] #num rows 540
xdim = blobs.shape[1] #num columns 960


# measure some skeleton metrics down here 
sum_local_neighbors = 0 #number of adjacent skeleton pixels 
sum_num_nodes = 0 #total number pixels that are of nodes of the skeleton
sum_num_edges = 0 #total number pixels that are of edges ots 
sum_num_endpoints = 0 #total number of pixels that are end points of the skeleton
sum_num_islands = 0 #total number of pixels that are isolated
sum_points_length = 0 #total number of skeleton pixels aka length of skeleton

total_num_edgelines = 0 #how many actual edges are there in the network

#lets make a bunch of arrays storing : node locations, endpoint locations, edge locations,island locations  
node_locations = []
edge_locations = []
endpoint_locations = []
island_locations = []

#here we are going to find the nodes and edges and endpoints

for row in range(0,ydim): #iterate through rows 
    for col in range(0,xdim): #iterate through columns
        
        if blobs[row][col] == True: #if pixel is skeleton add 1 to length of skeleton
            sum_points_length +=1
            
            #find the sum of the local neighbors included in the skeleton (including self): 1 = solo ,2= endpoint, 3 = edge , 4+ = node 
            for i in range(-1,2):
                for j in range(-1,2):
                    try: #this because there may be points on the edge of the picture
                        sum_local_neighbors += int(blobs[row +i][col +j])
                    except:
                        pass
            
            #figure out num nodes endpoints and edges
            if sum_local_neighbors >= 4: #if node
                node_locations.append([row,col])
                sum_num_nodes += 1
                total_num_edgelines += sum_local_neighbors -1 #count connection points (-1 to remove self counting)
            elif sum_local_neighbors == 3: # if edge
                edge_locations.append([row,col])
                sum_num_edges += 1
            elif sum_local_neighbors == 2: # if endpoint
                endpoint_locations.append([row,col])
                sum_num_endpoints += 1
                total_num_edgelines += sum_local_neighbors -1 #count connection points (-1 to remove self counting)
            else: #if island 
                island_locations.append([row,col])
                print("isolated point at ", blobs[row][col])
                sum_num_islands +=1

            #reset counter     
            sum_local_neighbors = 0

#error check
if sum_num_nodes + sum_num_edges + sum_num_endpoints + sum_num_islands == sum_points_length:
    print("all good")
else:
    print("f*&!")

total_num_edgelines = total_num_edgelines / 2 #because each edge has two endpoints 
print("this skeleton has {0} edge lines, {1} nodes, {2} endpoints, and {3} islands. \nthe skeleton is of length {4} pixels.".format(total_num_edgelines , sum_num_nodes, sum_num_endpoints, sum_num_islands, sum_points_length))

print("the edgelines are of average length {0}, .".format( sum_num_edges / total_num_edgelines))

print(node_locations)

(403, 341)
all good
this skeleton has 315.0 edge lines, 183 nodes, 60 endpoints, and 0 islands. 
the skeleton is of length 3177 pixels.
the edgelines are of average length 9.314285714285715, .
[[5, 118], [5, 119], [6, 79], [6, 118], [7, 78], [7, 79], [18, 115], [20, 49], [24, 107], [24, 108], [25, 107], [25, 211], [33, 28], [34, 27], [34, 28], [39, 222], [39, 223], [40, 45], [40, 222], [45, 106], [46, 105], [46, 106], [46, 107], [47, 43], [48, 42], [48, 43], [50, 225], [50, 271], [50, 272], [51, 272], [52, 228], [52, 229], [53, 229], [58, 212], [58, 213], [58, 214], [58, 234], [59, 213], [69, 137], [70, 137], [70, 138], [70, 188], [70, 244], [71, 243], [71, 244], [76, 259], [76, 260], [77, 260], [133, 98], [134, 97], [134, 98], [155, 241], [156, 241], [156, 242], [156, 309], [162, 113], [162, 133], [163, 132], [163, 133], [169, 108], [177, 216], [178, 216], [178, 217], [181, 214], [182, 214], [182, 215], [183, 313], [184, 312], [185, 240], [185, 241], [185, 242], [185, 311], [185, 312]